## Create Plots of Channel Conveyance Change

This notebook creates plots of channel conveyance change that pop up as a 'tool tip' when a user scroll

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import datetime as dt
import matplotlib as mpl
from datetime import timedelta
from scipy import stats
# sns.set()

fttom = 0.3048

In [68]:
gage_info = pd.read_csv(r'study_data\gauge_results.csv')

In [55]:
def trend_conf_int(x, y):
    x = x
    y = y
    lsq_res = stats.linregress(x, y)
    pred_y = lsq_res[1] + lsq_res[0] * x
    pred_x = np.linspace(np.min(x),np.max(x), len(x))
    return(pred_x, pred_y, lsq_res[0], lsq_res[2]*lsq_res[2], lsq_res[3])

In [73]:
for i, gage in gage_info.iterrows():
    gagename = gage['USGS Gagename']
    gageno = gage['USGS Gageno']
    channel_data_fn = r'study_data\channel_conveyance_change\%s_%s_channel_capacity_change.csv'%(gagename, gageno)
    channel_data = pd.read_csv(channel_data_fn)
    
    channel_data['DateTime'] = pd.to_datetime(channel_data.DateTime)
    channel_data['mean']= channel_data['q_resid_cms'].rolling(5).mean()

    f, ax = plt.subplots(figsize=(6,2))
    ax.scatter(
        channel_data['DateTime'],
        channel_data['q_resid_cms'],
        c='tomato',
        marker='o',
        s=10
    )
    if gage.category in ['minimal', 'monotonic']:
        pred_x, channel_data['pred_fit'], slope, r2, pval =  trend_conf_int(
            channel_data.DateTime.map(dt.datetime.toordinal),
            channel_data.q_resid_cms
        )
        ax.plot(
            channel_data.DateTime,
            channel_data.pred_fit,
            c='tomato',
            linestyle='--',
            linewidth=2
        )
    else:
        ax.plot(
            channel_data['DateTime'],
            channel_data['mean'],
            c='tomato',
            linestyle='--',
            linewidth=2
        )
    ax.set_ylabel('conveyance \n change '+ r'$(m^3/s)$', fontsize=13)
    ax.set_xlabel('date', fontsize=14)
    ax.set_title('%s (%s)'%(gagename, gageno), fontsize=16)
    ax.set_xlim(pd.to_datetime('01-01-1940'), pd.to_datetime('01-01-2020'))
    plt.xticks(rotation=40, fontsize=12)
    plt.yticks(fontsize=12)
    plt.savefig(r'figures\channel_conveyance_change\%s_%s.png'%(gagename, gageno),
                dpi=300,
                bbox_inches='tight')
    plt.close()